In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [11]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()
gmaps.configure(api_key=g_key)
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [3]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)


In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [15]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,Kahului,20.8947,-156.4700,83.97,72,40,19.57,US,2021-08-10 02:46:48
8,8,Butaritari,3.0707,172.7902,82.99,66,25,11.54,KI,2021-08-10 02:46:48
13,13,Vernon,50.2581,-119.2691,80.98,45,85,1.14,CA,2021-08-10 02:44:55
16,16,Walla Walla,46.0646,-118.3430,83.17,36,1,1.01,US,2021-08-10 02:46:50
18,18,San Andres,12.5847,-81.7006,82.42,78,75,10.36,CO,2021-08-10 02:46:51
25,25,Jasper,37.2001,-94.3502,85.23,78,1,1.01,US,2021-08-10 02:46:53
26,26,Georgetown,5.4112,100.3354,84.13,85,40,1.01,MY,2021-08-10 02:46:53
32,32,Quepos,9.4236,-84.1652,83.50,85,100,1.05,CR,2021-08-10 02:46:55
38,38,Curup,-3.4667,102.5333,78.44,75,96,2.15,ID,2021-08-10 02:46:56
43,43,Jamestown,42.0970,-79.2353,77.72,73,1,5.75,US,2021-08-10 02:42:01


In [18]:
preferred_cities_df.count()



City_ID       201
City          201
Lat           201
Lng           201
Max Temp      201
Humidity      201
Cloudiness    201
Wind Speed    201
Country       201
Date          201
dtype: int64

In [19]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Kahului,US,83.97,20.8947,-156.4700,
8,Butaritari,KI,82.99,3.0707,172.7902,
13,Vernon,CA,80.98,50.2581,-119.2691,
16,Walla Walla,US,83.17,46.0646,-118.3430,
18,San Andres,CO,82.42,12.5847,-81.7006,
25,Jasper,US,85.23,37.2001,-94.3502,
26,Georgetown,MY,84.13,5.4112,100.3354,
32,Quepos,CR,83.50,9.4236,-84.1652,
38,Curup,ID,78.44,-3.4667,102.5333,
43,Jamestown,US,77.72,42.0970,-79.2353,


In [ ]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")